# Flights Data Exploration Challenge

In this challenge, you'll explore a real-world dataset containing flights data from the US Department of Transportation.

Let's start by loading and viewing the data.

In [ ]:
#: Cargar los Datos
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


df_flights = pd.read_csv('flights.csv')
df_flights.head()


The dataset contains observations of US domestic flights in 2013, and consists of the following fields:

- **Year**: The year of the flight (all records are from 2013)
- **Month**: The month of the flight
- **DayofMonth**: The day of the month on which the flight departed
- **DayOfWeek**: The day of the week on which the flight departed - from 1 (Monday) to 7 (Sunday)
- **Carrier**: The two-letter abbreviation for the airline.
- **OriginAirportID**: A unique numeric identifier for the departure aiport
- **OriginAirportName**: The full name of the departure airport
- **OriginCity**: The departure airport city
- **OriginState**: The departure airport state
- **DestAirportID**: A unique numeric identifier for the destination aiport
- **DestAirportName**: The full name of the destination airport
- **DestCity**: The destination airport city
- **DestState**: The destination airport state
- **CRSDepTime**: The scheduled departure time
- **DepDelay**: The number of minutes departure was delayed (flight that left ahead of schedule have a negative value)
- **DelDelay15**: A binary indicator that departure was delayed by more than 15 minutes (and therefore considered "late")
- **CRSArrTime**: The scheduled arrival time
- **ArrDelay**: The number of minutes arrival was delayed (flight that arrived ahead of schedule have a negative value)
- **ArrDelay15**: A binary indicator that arrival was delayed by more than 15 minutes (and therefore considered "late")
- **Cancelled**: A binary indicator that the flight was cancelled

Your challenge is to explore the flight data to analyze possible factors that affect delays in departure or arrival of a flight.

1. Start by cleaning the data.
    R - Identify any null or missing data, and impute appropriate replacement values.
    R - Identify and eliminate any outliers in the **DepDelay** and **ArrDelay** columns.
2. Explore the cleaned data.
    R - View summary statistics for the numeric fields in the dataset.
    R - Determine the distribution of the **DepDelay** and **ArrDelay** columns.
    - Use statistics, aggregate functions, and visualizations to answer the following questions:
        R - *What are the average (mean) departure and arrival delays?*
        R - *How do the carriers compare in terms of arrival delay performance?*
        R - *Is there a noticable difference in arrival delays for different days of the week?*
        R - *Which departure airport has the highest average departure delay?*
        * - *Do **late** departures tend to result in longer arrival delays than on-time departures?*
        * - *Which route (from origin airport to destination airport) has the most **late** arrivals?*
        R - *Which route has the highest average arrival delay?*
        
Add markdown and code cells as required to create your solution.


# 1. Limpieza de Datos

#### 1.1  Cualquier dato nulo o faltante y reemplázalo con valores adecuados.

In [ ]:
# Identificar datos nulos
null_data = df_flights.isnull().sum()
print(null_data)

# Reemplazar valores nulos con el promedio (puedes ajustar el método de reemplazo según los campos)
df_flights_clean = df_flights.fillna(df_flights.mean())


#### 1.2 Identifica y elimina cualquier valor atípico en las columnas DepDelay y ArrDelay.

In [ ]:
# Identificar y eliminar outliers en DepDelay y ArrDelay usando el rango intercuartílico (IQR)
Q1_dep = df_flights_clean['DepDelay'].quantile(0.25)
Q3_dep = df_flights_clean['DepDelay'].quantile(0.75)
IQR_dep = Q3_dep - Q1_dep

Q1_arr = df_flights_clean['ArrDelay'].quantile(0.25)
Q3_arr = df_flights_clean['ArrDelay'].quantile(0.75)
IQR_arr = Q3_arr - Q1_arr

# Filtrar datos eliminando los valores atípicos
df_flights_clean = df_flights_clean[~((df_flights_clean['DepDelay'] < (Q1_dep - 1.5 * IQR_dep)) | (df_flights_clean['DepDelay'] > (Q3_dep + 1.5 * IQR_dep)))]
df_flights_clean = df_flights_clean[~((df_flights_clean['ArrDelay'] < (Q1_arr - 1.5 * IQR_arr)) | (df_flights_clean['ArrDelay'] > (Q3_arr + 1.5 * IQR_arr)))]


# 2. Explora los datos limpios:

#### 2.1 Visualiza estadísticas descriptivas para los campos numéricos del conjunto de datos.

In [ ]:
# Estadísticas descriptivas
print(df_flights_clean.describe())


#### 2.2 Determina la distribución de las columnas DepDelay y ArrDelay.

In [ ]:
# Histograma para DepDelay
plt.figure(figsize=(10, 5))
sns.histplot(df_flights_clean['DepDelay'], bins=50, kde=True, color='blue')
plt.title('Distribución de Retrasos en la Salida (DepDelay)')
plt.xlabel('Retraso en Minutos')
plt.ylabel('Frecuencia')
plt.show()

# Histograma para ArrDelay
plt.figure(figsize=(10, 5))
sns.histplot(df_flights_clean['ArrDelay'], bins=50, kde=True, color='green')
plt.title('Distribución de Retrasos en la Llegada (ArrDelay)')
plt.xlabel('Retraso en Minutos')
plt.ylabel('Frecuencia')
plt.show()

# Boxplot para DepDelay
plt.figure(figsize=(8, 4))
sns.boxplot(x=df_flights_clean['DepDelay'], color='blue')
plt.title('Boxplot de Retrasos en la Salida (DepDelay)')
plt.show()

# Boxplot para ArrDelay
plt.figure(figsize=(8, 4))
sns.boxplot(x=df_flights_clean['ArrDelay'], color='green')
plt.title('Boxplot de Retrasos en la Llegada (ArrDelay)')
plt.show()


# 3 Usa estadísticas, funciones de agregación y visualizaciones para responder a las siguientes preguntas

#### 3.1 ¿Cuáles son los retrasos promedio (media) de salida y llegada?

In [ ]:
# Media de retrasos
mean_dep_delay = df_flights_clean['DepDelay'].mean()
mean_arr_delay = df_flights_clean['ArrDelay'].mean()

print(f"Retraso promedio de salida: {mean_dep_delay:.2f} minutos")
print(f"Retraso promedio de llegada: {mean_arr_delay:.2f} minutos")


#### 3.2 ¿Cómo se comparan las aerolíneas en cuanto a rendimiento de retraso en la llegada?

In [ ]:
# Comparación de aerolíneas por retraso en la llegada
carrier_arr_delay = df_flights_clean.groupby('Carrier')['ArrDelay'].mean().sort_values()

# Visualización
plt.figure(figsize=(10, 6))
sns.barplot(x=carrier_arr_delay.index, y=carrier_arr_delay.values, palette='viridis')
plt.title('Retraso Promedio de Llegada por Aerolínea')
plt.xlabel('Aerolínea')
plt.ylabel('Retraso Promedio en Minutos')
plt.show()


#### 3.3 ¿Hay una diferencia notable en los retrasos de llegada según el día de la semana?

In [ ]:
# Retraso en la llegada según el día de la semana
weekday_arr_delay = df_flights_clean.groupby('DayOfWeek')['ArrDelay'].mean()

# Visualización
plt.figure(figsize=(10, 6))
sns.barplot(x=weekday_arr_delay.index, y=weekday_arr_delay.values, palette='coolwarm')
plt.title('Retraso Promedio de Llegada por Día de la Semana')
plt.xlabel('Día de la Semana (1= Lunes, 7= Domingo)')
plt.ylabel('Retraso Promedio en Minutos')
plt.show()


#### 3.4 ¿Qué aeropuerto de salida tiene el mayor retraso promedio de salida?

In [ ]:
# Retraso promedio de salida por aeropuerto
origin_dep_delay = df_flights_clean.groupby('OriginAirportName')['DepDelay'].mean().sort_values(ascending=False)

# Mostrar el aeropuerto con mayor retraso
print(f"El aeropuerto de salida con mayor retraso promedio es {origin_dep_delay.index[0]} con un retraso promedio de {origin_dep_delay.values[0]:.2f} minutos")


#### 3.5 ¿Las salidas tardías tienden a provocar mayores retrasos en la llegada que las salidas a tiempo?

In [ ]:
# Comparación entre vuelos que salieron a tiempo y vuelos que salieron tarde
on_time = df_flights_clean[df_flights_clean['DepDelay'] <= 0]['ArrDelay'].mean()
late_departures = df_flights_clean[df_flights_clean['DepDelay'] > 0]['ArrDelay'].mean()

print(f"Retraso promedio de llegada para vuelos a tiempo: {on_time:.2f} minutos")
print(f"Retraso promedio de llegada para vuelos con salida tardía: {late_departures:.2f} minutos")


#### 3.6 ¿Qué ruta (del aeropuerto de origen al aeropuerto de destino) tiene la mayor cantidad de llegadas tarde?

In [ ]:
# Rutas con mayor número de llegadas tarde
routes_late_arrivals = df_flights_clean[df_flights_clean['ArrDelay15'] == 1].groupby(['OriginAirportName', 'DestAirportName']).size().sort_values(ascending=False)

# Mostrar la ruta con mayor número de llegadas tarde
print(f"La ruta con más llegadas tarde es de {routes_late_arrivals.index[0][0]} a {routes_late_arrivals.index[0][1]} con {routes_late_arrivals.values[0]} llegadas tarde")


#### 3.7 ¿Qué ruta tiene el mayor retraso promedio de llegada?

In [ ]:
# Rutas con mayor retraso promedio en la llegada
routes_avg_arr_delay = df_flights_clean.groupby(['OriginAirportName', 'DestAirportName'])['ArrDelay'].mean().sort_values(ascending=False)

# Mostrar la ruta con mayor retraso promedio de llegada
print(f"La ruta con mayor retraso promedio de llegada es de {routes_avg_arr_delay.index[0][0]} a {routes_avg_arr_delay.index[0][1]} con un retraso promedio de {routes_avg_arr_delay.values[0]:.2f} minutos")
